In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# параметры распределений
mu0, cov0 = [-2.0, 3.0], [[1.0, 0.2], [0.3, 1.0]]
mu1, cov1 = [3.0, 2.0], [[1.0, 0.0], [0.0, 1.0]]

In [3]:
def sample(d0, d1, n=32):
    x0 = d0.sample((n,))
    x1 = d1.sample((n,))
    y0 = torch.zeros((n, 1))
    y1 = torch.ones((n, 1))
    return torch.cat([x0, x1], 0), torch.cat([y0, y1], 0)

In [4]:
d0 = torch.distributions.MultivariateNormal(torch.tensor(mu0), torch.tensor(cov0))
d1 = torch.distributions.MultivariateNormal(torch.tensor(mu1), torch.tensor(cov1))

In [5]:
layer = nn.Linear(2, 1)
print([p.data[0] for p in layer.parameters()])
opt = optim.SGD(lr=1e-7, params=list(layer.parameters()))

[tensor([-0.0723,  0.4233]), tensor(-0.5142)]


In [6]:
log_freq = 500
for i in range(10000):
    if i%log_freq == 0:
        with torch.no_grad():
            x, y = sample(d0, d1, 100000)
            out = F.sigmoid(layer(x))
            loss = F.binary_cross_entropy(out, y)
        print('Ошибка после %d итераций: %f' %(i/log_freq, loss))
    x, y = sample(d0, d1, 1024)
    out = F.sigmoid(layer(x))
    loss = F.binary_cross_entropy(out, y)
    loss.backward()
    opt.step()


Ошибка после 0 итераций: 0.957849
Ошибка после 1 итераций: 0.927825
Ошибка после 2 итераций: 0.841064
Ошибка после 3 итераций: 0.712904
Ошибка после 4 итераций: 0.564393
Ошибка после 5 итераций: 0.420819
Ошибка после 6 итераций: 0.302686
Ошибка после 7 итераций: 0.214006
Ошибка после 8 итераций: 0.153259
Ошибка после 9 итераций: 0.113440
Ошибка после 10 итераций: 0.085805
Ошибка после 11 итераций: 0.068076
Ошибка после 12 итераций: 0.055082
Ошибка после 13 итераций: 0.046425
Ошибка после 14 итераций: 0.040355
Ошибка после 15 итераций: 0.035297
Ошибка после 16 итераций: 0.032754
Ошибка после 17 итераций: 0.030681
Ошибка после 18 итераций: 0.028217
Ошибка после 19 итераций: 0.027496


In [ ]:
#x_scale = np.linspace(-10, 10, 5000)
#d0_pdf = stats.norm.pdf(x_scale, mu0, sigma0) 
#d1_pdf = stats.norm.pdf(x_scale, mu1, sigma1)
#x_tensor = torch.tensor(x_scale.reshape(-1, 1), dtype=torch.float)
#with torch.no_grad():
#    dist = F.sigmoid(layer(x_tensor)).numpy()

In [ ]:
#plt.plot(x_scale, d0_pdf*2, label='d0') # умножение на 2 для красоты графиков, на распределения не влияет
#plt.plot(x_scale, d1_pdf*2, label='d1')
#plt.plot(x_scale, dist.flatten(), label='pred')
#plt.legend();

In [ ]:
#opt = optim.SGD(lr=1e-7, params=list(layer.parameters()))